## Section 3.1: Extract data from Yahoo! Finance page

 - Load the yfinance module
 - Scan `Yahoo! Finance` page for the stock symbols that appeared in the news
 - Capture/retrieve the `stock price` and `volume` for `last 30 days`.

In [2]:
import pandas as pd
from datetime import date, timedelta

In [3]:
#Loading yfinance module and installing it if it doesn't already exist
try:
    success_msg = 'Yahoo! Finance module loaded.'
    import yfinance as yf
    print(success_msg)
except ModuleNotFoundError as e:
    print('Installing Yahoo! Finance python module.\n')
    !pip install yfinance
    import yfinance as yf
    print(success_msg)

Yahoo! Finance module loaded.


In [4]:
df = pd.read_csv('./Outputfile.csv')

In [5]:
df.head()

,Date,Title,Ticker,Article URL
0,2021-11-27 12:14:00,Tendances crypto dans l'iGaming : aperçus de ...,NaN,https://www.prnewswire.com/news-releases/tenda...
1,2021-11-27 12:09:00,Krypto-Trends im iGaming: SOFTSWISS Insights,NaN,https://www.prnewswire.com/news-releases/krypt...
2,2021-11-27 11:13:00,GAC MOTOR comemora um ano no Chile,NaN,https://www.prnewswire.com/news-releases/gac-m...
3,2021-11-27 11:00:00,BlueVine Announces U.S. Small Business Resili...,NaN,https://www.prnewswire.com/news-releases/bluev...
4,2021-11-27 09:30:00,Secondary Market Research Sourcing and Procur...,NaN,https://www.prnewswire.com/news-releases/secon...


In [6]:
 num_of_stocks = int(input("How many stocks do you want to analyze : "))

How many stocks do you want to analyze : 2


In [7]:
tickers = df['Ticker'].dropna()
top_tickers = tickers.value_counts()[:num_of_stocks].index.tolist()
print(top_tickers)

['CTXS', 'CPLG']


In [14]:
stock_details = []

for tick in top_tickers:
    stock = yf.Ticker(tick)
    stock_details.append(stock)
    
    temp_stock = yf.download(tick, start=(date.today()-timedelta(days=30)).isoformat(), end=date.today())
    base = date.today()
    date_list = [base - timedelta(days=x) for x in range(30)]
    temp_stock['date'] = date_list
    temp_stock.to_csv(tick+'.csv', index=False)
    


[*********************100%***********************]  1 of 1 completed
Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')
[*********************100%***********************]  1 of 1 completed
Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')
